# torch.Tensor.register_hook(hook)  
功能：注册一个反向传播hook函数，这个函数是Tensor类里的，当计算tensor的梯度时自动执行。   
形式： hook(grad) -> Tensor or None ，其中grad就是这个tensor的梯度。  
返回值：a handle that can be used to remove the added hook by calling handle.remove() 

## 例1

In [1]:
import torch
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)
a = torch.add(w, x)  # retain_grad()
b = torch.add(w, 1)
y = torch.mul(a, b)
a_grad = list()
def grad_hook(grad):
    a_grad.append(grad)

handle = a.register_hook(grad_hook)

y.backward()

# 查看梯度
print("gradient:", w.grad, x.grad, a.grad, b.grad, y.grad)
print("a_grad[0]: ", a_grad[0])
handle.remove()

gradient: tensor([5.]) tensor([2.]) None None None
a_grad[0]:  tensor([2.])


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27296\1321482932.py:16: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:494.)
  print("gradient:", w.grad, x.grad, a.grad, b.grad, y.grad)


## 例2

In [3]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)
a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)
a_grad = list()
def grad_hook(grad):
    grad *= 2
    # return grad*3
handle = w.register_hook(grad_hook)
y.backward()

# 查看梯度
print("w.grad: ", w.grad)
handle.remove()

w.grad:  tensor([10.])


# torch.nn.Module.register_forward_hook
功能：Module前向传播中的hook,module在前向传播后，自动调用hook函数。  
形式：hook(module, input, output) -> None。注意不能修改input和output的返回值。
其中，module是当前网络层，input是网络层的输入数据, output是网络层的输出数据

应用场景：如用于提取特征图  
举例：假设网络由卷积层conv1和池化层pool1构成，输入一张4\*4的图片，现采用forward_hook获取module——conv1之后的feature maps，示意图如下：

![](hook-demo.png)

In [3]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        return x
def farward_hook(module, data_input, data_output):
    fmap_block.append(data_output)
    input_block.append(data_input)
    
if __name__ == "__main__":
    # 初始化网络
    net = Net()
    net.conv1.weight[0].fill_(1)
    net.conv1.weight[1].fill_(2)
    net.conv1.bias.data.zero_()
    
    # 注册hook
    fmap_block = list()
    input_block = list()
    net.conv1.register_forward_hook(farward_hook)
    
    # inference
    fake_img = torch.ones((1, 1, 4, 4))   # batch size * channel * H * W
    output = net(fake_img)
    
    # 观察
    print("output shape: {}\noutput value: {}\n".format(output.shape, output))
    print("feature maps shape: {}\noutput value: {}\n".format(fmap_block[0].shape, fmap_block[0]))
    print("input shape: {}\ninput value: {}".format(input_block[0][0].shape, input_block[0]))

RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.